# Tests for Normaldistribution (glm_norm)

In [1]:
import os
import datetime
import numpy as np
import pprint

import logging
import warnings

%load_ext autoreload
%autoreload 2

In [2]:
import batchglm.api as glm

np.warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")
logging.getLogger("tensorflow").setLevel(logging.INFO)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

## Import batchglm

In [3]:
#import batchglm.api as glm

## Sample from Normal Distribution

In [4]:
norm = glm.utils.random.Normal(500, 0.7)

In [5]:
norm.sample(10)

array([500.41459249, 499.85053908, 500.2277094 , 499.92377473,
       500.21808949, 499.7723035 , 500.24325824, 500.00966689,
       500.38119543, 499.59979689])

## Simulate some Data

In [6]:
sim = glm.models.glm_norm.Simulator()

In [7]:
sim.generate()

In [8]:
sim.X

<xarray.DataArray 'X' (observations: 1000, features: 100)>
array([[479.278305, 505.504743, 500.661207, ..., 552.004008, 478.499959,
        481.360518],
       [480.612642, 504.515243, 501.30954 , ..., 550.651545, 479.695945,
        482.305405],
       [478.803265, 504.773341, 502.322371, ..., 552.485536, 478.281363,
        480.885772],
       ...,
       [482.38018 , 505.777565, 504.810279, ..., 552.740605, 481.897705,
        481.494682],
       [480.679676, 504.216359, 503.805206, ..., 550.906321, 478.391097,
        481.525515],
       [479.784229, 506.843516, 503.485235, ..., 553.766184, 478.984799,
        482.301796]])
Dimensions without coordinates: observations, features

In [9]:
sim.X - sim.mean

<xarray.DataArray (observations: 1000, features: 100)>
array([[ 0.278305,  1.504743, -1.338793, ...,  1.004008,  0.499959,  0.360518],
       [ 0.536107, -0.521418, -1.733235, ..., -1.251504,  0.766803,  0.34975 ],
       [-0.196735,  0.773341,  0.322371, ...,  1.485536,  0.281363, -0.114228],
       ...,
       [ 1.323922, -0.343836,  0.711075, ..., -0.196516,  1.978326, -1.428857],
       [ 0.699953, -0.868381,  0.748777, ..., -1.127752, -0.59914 , -0.44237 ],
       [-1.272028,  0.722115, -0.61397 , ...,  0.829063, -0.93458 , -0.621744]])
Dimensions without coordinates: observations, features

In [10]:
sim.par_link_loc

<xarray.DataArray (design_loc_params: 5, features: 100)>
array([[479.      , 504.      , 502.      , ..., 551.      , 478.      ,
        481.      ],
       [  1.076535,   1.036661,   1.042776, ...,   0.903048,   0.929142,
          0.955655],
       [  1.064394,   1.017214,   0.924213, ...,   1.041112,   0.904225,
          0.939179],
       [  0.97114 ,   0.936083,   1.033764, ...,   0.983129,   1.013412,
          1.018328],
       [  0.979723,   1.08474 ,   1.056429, ...,   1.034072,   0.990236,
          0.967884]])
Coordinates:
  * design_loc_params  (design_loc_params) object 'Intercept' ... 'batch[T.3]'
Dimensions without coordinates: features

In [11]:
sim.par_link_scale

<xarray.DataArray (design_scale_params: 5, features: 100)>
array([[-0.039111, -0.029147,  0.05541 , ...,  0.061251, -0.087802, -0.088112],
       [-0.052688,  0.055862,  0.018404, ..., -0.093762, -0.059667, -0.076306],
       [ 0.058475,  0.05075 ,  0.035833, ...,  0.081699, -0.099882, -0.02032 ],
       [-0.052368, -0.003401,  0.018385, ...,  0.014725, -0.016392,  0.034   ],
       [ 0.091344,  0.015848, -0.01487 , ..., -0.059769,  0.006367,  0.027399]])
Coordinates:
  * design_scale_params  (design_scale_params) object 'Intercept' ... 'batch[T.3]'
Dimensions without coordinates: features

## Set up Estimator

In [12]:
X = sim.X
design_loc = sim.design_loc
design_scale = sim.design_scale

# input data
input_data = glm.models.glm_norm.InputData.new(data=X, design_loc=design_loc, design_scale=design_scale)

In [13]:
estimator = glm.models.glm_norm.Estimator(input_data)
#estimator.initialize()

AttributeError: 'BasicModelGraph' object has no attribute 'mu'